# Deploy the Application Package

Deploying an Application Package using the OGC API Processes API uses the API resource highlighted in the table below:


| **Resource**                   | **Path**                                  | **Purpose**                                                                     | **Part**   |
|--------------------------------|-------------------------------------------|---------------------------------------------------------------------------------|------------|
| Landing page                   | `/`                                       | Top-level resource serving as an entry point.                                   | Part 1     |
| Conformance declaration        | `/conformance`                            | Information about the functionality supported by the server.                    | Part 1     |
| API Definition                 | `/api`                                    | Metadata about the API itself.                                                  | Part 1     |
| Process list                   | `/processes`                              | Lists available processes with identifiers and links to descriptions.           | Part 1     |
| Process description            | `/processes/{processID}`                  | Retrieves detailed information about a specific process.                        | Part 1     |
| Process execution              | `/processes/{processID}/execution`        | Executes a process, creating a job.                                             | Part 1     |
| **Deploy Process**             | **`/processes` (POST)**                   | **Deploys a new process on the server.**                                        | **Part 2** |
| Replace Process                | `/processes/{processID}` (PUT)            | Replaces an existing process with a new version.                                | Part 2     |
| Undeploy Process               | `/processes/{processID}` (DELETE)         | Removes an existing process from the server.                                    | Part 2     |
| Job status info                | `/jobs/{jobID}`                           | Retrieves the current status of a job.                                          | Part 1     |
| Job results                    | `/jobs/{jobID}/results`                   | Retrieves the results of a job.                                                 | Part 1     |
| Job list                       | `/jobs`                                   | Retrieves a list of submitted jobs.                                             | Part 1     |
| Job deletion                   | `/jobs/{jobID}`                           | Cancels and deletes a job.                                                      | Part 1     |


This resource permits the deployment of the an Application Package and provide two options for the `Content-Type`.

## Hands-on - Deploy the water_bodies Application Package

**Encoding Options**

The deployment can use two encodings that are based on the same CWL conformance class. Both methods utilize the same water_bodies.cwl file, but the way the file is provided differs:
- OGC Application Package Encoding (application/ogcapppkg+json): This method allows you to reference the CWL file by providing its location, rather than including the file's content in the request.
- CWL Encoding (application/cwl+yaml): This method requires the CWL file content to be included directly in the request body.

**Request Configuration**

When selecting a content type, the request body text area updates to contain a relevant payload template for that encoding.
Warning: If the payload is manually edited, switching to a different encoding may not refresh the text area. In this case, the Reset button can be used to restore the appropriate template.

**Server Response**

After executing the deployment request, the server responds with a process summary similar to the one obtained from the previous process listing endpoint.
The server’s response includes a Location header containing the URL to access the detailed process description.

**Next Steps** 

You can either:
- Return to the list of available processes to verify the newly deployed process.
- Proceed to the next step to review the process description in detail

## Setup

Lists available processes in the `acme` namespace.

> **_NOTE:_**: if the `acme` namespace does not exist, ZOO Project will create it.

In [13]:
import requests
import json
import yaml
import time
from loguru import logger
from pystac.item_collection import ItemCollection

namespace = "acme"

ogc_api_endpoint = f"http://zoo-project-dru-service/{namespace}/ogc-api"

r = requests.get(f"{ogc_api_endpoint}/processes")

r.status_code

200


If the application package was deployed previously, delete it.

In [14]:
def undeploy(process_id):

    r = requests.delete(f"{ogc_api_endpoint}/processes/{process_id}")

    return r

app_package_entry_point = "main"

undeploy(app_package_entry_point)

r.status_code

200

## OGC Application Package Encoding (application/ogcapppkg+json)

In [15]:
headers = {"accept": "application/json", 
          "Content-Type": "application/ogcapppkg+json"}

app_package_entry_point = "main"

response = requests.post(
    
    f"{ogc_api_endpoint}/processes/?w={app_package_entry_point}",
    headers=headers,
    json = {
      "executionUnit": {
        "href": "https://github.com/eoap/mastering-app-package/releases/download/1.0.0/app-water-bodies-cloud-native.1.0.0.cwl",
        "type": "application/cwl"
      }
    }
)

In [24]:
response.json()

{'id': 'main',
 'title': 'Water bodies detection based on NDWI and otsu threshold',
 'description': 'Water bodies detection based on NDWI and otsu threshold applied to Sentinel-2 COG STAC items',
 'mutable': True,
 'version': '1.0.0',
 'metadata': [{'role': 'https://schema.org/softwareVersion', 'value': '1.0.0'},
  {'role': 'https://schema.org/author',
   'value': {'@context': 'https://schema.org',
    '@type': 'Person',
    's.name': 'Jane Doe',
    's.email': 'jane.doe@acme.earth',
    's.affiliation': 'ACME'}}],
 'outputTransmission': ['value', 'reference'],
 'jobControlOptions': ['async-execute', 'dismiss'],
 'links': [{'rel': 'http://www.opengis.net/def/rel/ogc/1.0/execute',
   'type': 'application/json',
   'title': 'Execute End Point',
   'href': 'http://localhost:8080/acme/ogc-api/processes/main/execution'}]}

In [23]:
dict(response.headers)

{'Date': 'Mon, 21 Oct 2024 13:06:57 GMT',
 'Server': 'Apache/2.4.41 (Ubuntu)',
 'X-Powered-By': 'ZOO-Project-DRU',
 'X-Also-Powered-By': 'jwt.securityIn',
 'X-Also-Also-Powered-By': 'dru.securityIn',
 'X-Also-Also-Also-Powered-By': 'dru.securityOut',
 'Location': 'http://localhost:8080/acme/ogc-api/processes/main',
 'Keep-Alive': 'timeout=5, max=100',
 'Connection': 'Keep-Alive',
 'Transfer-Encoding': 'chunked',
 'Content-Type': 'application/json;charset=UTF-8'}

## CWL Encoding (application/cwl+yaml)

If the application package was deployed previously, delete it.

In [26]:
app_package_entry_point = "main"

undeploy(app_package_entry_point)

<Response [204]>

Download the application package from https://github.com/eoap/mastering-app-package/releases/download/1.0.0/app-water-bodies-cloud-native.1.0.0.cwl

In [27]:
r = requests.get("https://github.com/eoap/mastering-app-package/releases/download/1.0.0/app-water-bodies-cloud-native.1.0.0.cwl")

app_package_content  = yaml.safe_load(r.content)

app_package_content

{'cwlVersion': 'v1.0',
 '$namespaces': {'s': 'https://schema.org/'},
 's:softwareVersion': '1.0.0',
 'schemas': ['http://schema.org/version/9.0/schemaorg-current-http.rdf'],
 '$graph': [{'class': 'Workflow',
   'id': 'main',
   'label': 'Water bodies detection based on NDWI and otsu threshold',
   'doc': 'Water bodies detection based on NDWI and otsu threshold applied to Sentinel-2 COG STAC items',
   'requirements': [{'class': 'ScatterFeatureRequirement'},
    {'class': 'SubworkflowFeatureRequirement'}],
   'inputs': {'aoi': {'label': 'area of interest',
     'doc': 'area of interest as a bounding box',
     'type': 'string'},
    'epsg': {'label': 'EPSG code',
     'doc': 'EPSG code',
     'type': 'string',
     'default': 'EPSG:4326'},
    'stac_items': {'label': 'Sentinel-2 STAC items',
     'doc': 'list of Sentinel-2 COG STAC items',
     'type': 'string[]'},
    'bands': {'label': 'bands used for the NDWI',
     'doc': 'bands used for the NDWI',
     'type': 'string[]',
     'def

In [28]:
app_package_entry_point = "main"

headers = {"accept": "application/json", 
          "Content-Type": "application/cwl+yaml"}

response = requests.post(
    f"{ogc_api_endpoint}/processes?w={app_package_entry_point}",
    headers=headers,
    json=app_package_content
)

response.status_code

201

In [29]:
response.json()

{'id': 'main',
 'title': 'Water bodies detection based on NDWI and otsu threshold',
 'description': 'Water bodies detection based on NDWI and otsu threshold applied to Sentinel-2 COG STAC items',
 'mutable': True,
 'version': '1.0.0',
 'metadata': [{'role': 'https://schema.org/softwareVersion', 'value': '1.0.0'},
  {'role': 'https://schema.org/author',
   'value': {'@context': 'https://schema.org',
    '@type': 'Person',
    's.name': 'Jane Doe',
    's.email': 'jane.doe@acme.earth',
    's.affiliation': 'ACME'}}],
 'outputTransmission': ['value', 'reference'],
 'jobControlOptions': ['async-execute', 'dismiss'],
 'links': [{'rel': 'http://www.opengis.net/def/rel/ogc/1.0/execute',
   'type': 'application/json',
   'title': 'Execute End Point',
   'href': 'http://localhost:8080/acme/ogc-api/processes/main/execution'}]}

In [30]:
dict(response.headers)

{'Date': 'Mon, 21 Oct 2024 13:11:01 GMT',
 'Server': 'Apache/2.4.41 (Ubuntu)',
 'X-Powered-By': 'ZOO-Project-DRU',
 'X-Also-Powered-By': 'jwt.securityIn',
 'X-Also-Also-Powered-By': 'dru.securityIn',
 'X-Also-Also-Also-Powered-By': 'dru.securityOut',
 'Location': 'http://localhost:8080/acme/ogc-api/processes/main',
 'Keep-Alive': 'timeout=5, max=100',
 'Connection': 'Keep-Alive',
 'Transfer-Encoding': 'chunked',
 'Content-Type': 'application/json;charset=UTF-8'}

**Next Steps** 

You can either:
- Return to the list of available processes to verify the newly deployed process.
- Proceed to the next step to review the process description in detail